In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)


import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)
cc.df

In [ ]:
cc.df = cc.df.loc[cc.df['uf_residencia']=='PR']

In [ ]:
casos = cc.df[['id_notifica','ibge_residencia','data_diagnostico']].join(municipios.set_index('ibge')[['municipio','rs','macro','populacao']],on='ibge_residencia')
casos['ano_caso'] = casos['data_diagnostico'].dt.year
del casos['data_diagnostico']

gcasos = casos.groupby(['ibge_residencia','municipio','rs','macro','populacao','ano_caso']).count().unstack('ano_caso').droplevel(0,1)
gcasos.columns.name = ''
gcasos.columns = pd.MultiIndex.from_product([['casos'],gcasos.columns])
gcasos

In [ ]:
obitos = cc.df.loc[cc.df['evolucao']==2,['id_notifica','ibge_residencia','data_cura_obito']].join(municipios.set_index('ibge')[['municipio','rs','macro','populacao']],on='ibge_residencia')
obitos['ano_obito'] = obitos['data_cura_obito'].dt.year

del obitos['data_cura_obito']

gobitos = obitos.groupby(['ibge_residencia','municipio','rs','macro','populacao','ano_obito']).count().unstack('ano_obito').droplevel(0,1)
gobitos.columns.name = ''
gobitos.columns = pd.MultiIndex.from_product([['obitos'],gobitos.columns])
gobitos

In [ ]:
casos_obitos = gcasos.join(gobitos).fillna(0).astype(int)
casos_obitos

In [ ]:
coeficientes = casos_obitos / casos_obitos.reset_index('populacao')[['populacao']].values * 100000
coeficientes = coeficientes.rename(columns={'casos':'coef. inc','obitos':'coef. mort'})
coeficientes

In [ ]:
tx_let = casos_obitos['obitos'] / casos_obitos['casos']
tx_let.columns = pd.MultiIndex.from_product([['tx. let'],tx_let.columns])
tx_let

In [ ]:
alll = casos_obitos.join(coeficientes).join(tx_let)
alll

In [ ]:
alll = alll.reorder_levels([1,0],axis=1).sort_index(axis=1)

In [ ]:
alll = alll.sort_values(['rs', 'municipio'])
alll.to_excel(join(default_output,'coeficientes_municipios.xlsx'))

In [ ]:
## POR RS

In [ ]:
casos_obitos_rs = casos_obitos.groupby(['rs', 'populacao']).sum().reset_index('populacao')
casos_obitos_rs = casos_obitos_rs.groupby(['rs']).sum()

casos_obitos_rs = casos_obitos_rs.reset_index('rs')
casos_obitos_rs = casos_obitos_rs.set_index(['rs', 'populacao'])
casos_obitos_rs

In [ ]:
coeficientes = casos_obitos_rs / casos_obitos_rs.reset_index('populacao')[['populacao']].values * 100000
coeficientes = coeficientes.rename(columns={'casos':'coef. inc','obitos':'coef. mort'})


tx_let = casos_obitos_rs['obitos'] / casos_obitos_rs['casos']
tx_let.columns = pd.MultiIndex.from_product([['tx. let'],tx_let.columns])

In [ ]:
alll = casos_obitos_rs.join(coeficientes).join(tx_let)
alll = alll.reorder_levels([1,0],axis=1).sort_index(axis=1)
alll.to_excel(join(default_output,'coeficientes_rs.xlsx'))

In [ ]:
## POR MACRO

In [ ]:
casos_obitos_macro = casos_obitos.groupby(['macro', 'populacao']).sum().reset_index('populacao')
casos_obitos_macro = casos_obitos_macro.groupby(['macro']).sum()

casos_obitos_macro = casos_obitos_macro.reset_index('macro')
casos_obitos_macro = casos_obitos_macro.set_index(['macro', 'populacao'])
casos_obitos_macro

In [ ]:
coeficientes = casos_obitos_macro / casos_obitos_macro.reset_index('populacao')[['populacao']].values * 100000
coeficientes = coeficientes.rename(columns={'casos':'coef. inc','obitos':'coef. mort'})


tx_let = casos_obitos_macro['obitos'] / casos_obitos_macro['casos']
tx_let.columns = pd.MultiIndex.from_product([['tx. let'],tx_let.columns])

In [ ]:
alll = casos_obitos_macro.join(coeficientes).join(tx_let)
alll = alll.reorder_levels([1,0],axis=1).sort_index(axis=1)
alll.to_excel(join(default_output,'coeficientes_macro.xlsx'))